# AI Bootcamp 02 - Code Generation Agent

In [1]:
import os
import json
from openai import OpenAI

In [2]:
# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "YOUR_API_KEY"))

In [3]:
class CodeGenerationAgent:
    def __init__(self, model="gpt-4-turbo", coding_guidelines=None):
        self.model = model
        self.coding_guidelines = coding_guidelines or [
            "Follow Python best practices (PEP8).",
            "Ensure the code is modular and well-documented.",
            "Use existing libraries where applicable.",
            "Provide unit tests for critical functions."
        ]

    def refine_request(self, user_request):
        """
        Uses OpenAI to refine the user request by asking clarifying questions.
        """
        system_prompt = (
            "You are a senior software engineer helping to clarify coding requirements. "
            "Ask questions to refine the user's request before generating code."
        )

        user_prompt = f"User request: {user_request}\nWhat clarifications do you need?"

        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.3,
            max_tokens=150
        )

        return response.choices[0].message.content.strip()

    def generate_code(self, user_request, additional_context=""):
        """
        Generates Python code based on user request and coding guidelines.
        """
        system_prompt = (
            "You are an AI coding assistant. Generate Python code based on user requirements while following best practices."
            f" Adhere to these guidelines:\n- " + "\n- ".join(self.coding_guidelines)
        )

        user_prompt = f"Task: {user_request}\n{additional_context}\nProvide the code:"

        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.2,
            max_tokens=500
        )

        return response.choices[0].message.content.strip()

    def validate_code(self, generated_code):
        """
        Reviews the generated code and provides suggestions for improvements.
        """
        system_prompt = (
            "You are an experienced software engineer reviewing code for best practices, efficiency, and correctness."
            " Suggest improvements and flag potential issues."
        )

        user_prompt = f"Review the following Python code:\n```python\n{generated_code}\n```"

        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.3,
            max_tokens=300
        )

        return response.choices[0].message.content.strip()

    def interact_with_user(self, user_request):
        """
        Full interaction cycle:
        1. Refine the request.
        2. Generate the code.
        3. Validate and suggest improvements.
        4. Return final output.
        """
        # Step 1: Clarify user request
        clarification = self.refine_request(user_request)
        print("\n[Clarification Needed]:", clarification)

        # Step 2: Generate initial code
        generated_code = self.generate_code(user_request, additional_context=clarification)
        print("\n[Generated Code]:\n", generated_code)

        # Step 3: Validate and improve
        review_feedback = self.validate_code(generated_code)
        print("\n[Code Review Feedback]:\n", review_feedback)

        return generated_code, review_feedback



In [4]:
# -------------------------- USAGE --------------------------
if __name__ == "__main__":
    agent = CodeGenerationAgent()

    # Example user request
    user_request = "Write a function in Python to sort a list using quicksort."

    # Interact with the agent
    final_code, feedback = agent.interact_with_user(user_request)

    print("\n[Final Code Output]:\n", final_code)
    print("\n[Final Feedback]:\n", feedback)



[Clarification Needed]: To clarify the user's request for implementing a quicksort function in Python, here are some questions that could help refine the requirements:

1. **Input Specification**: 
   - What types of elements should the function be able to sort? Are we only sorting integers, or should it handle other data types like floats or strings?
   - Should the function handle lists with mixed data types?

2. **Output Specification**:
   - Should the function return a new sorted list, or should it sort the list in place?

3. **Performance and Optimization**:
   - Are there any specific performance constraints or expectations, such as the size of the input list?
   - Is there a specific variant of quicksort (like three-way partitioning) that

[Generated Code]:
 To implement a quicksort function in Python, I'll assume the following based on typical use cases:

1. **Input Specification**:
   - The function will be able to sort lists of integers, floats, and strings. However, it wil